# Date Extraction from Medical Notes

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd
import numpy as np
doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [2]:
df = df.to_frame().rename(columns={0:'Text'})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 1 columns):
Text    500 non-null object
dtypes: object(1)
memory usage: 4.0+ KB


## 1. Extract Dates Formatted Month/Day/Year, Month/Year

### Example:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* 6/2008; 12/2009

### Pattern: 
   - No \[word character or '/'\] precedes the string
   2. ((1-2 digits)'/') appears once or twice, indicating either Month/Day or Month
   3. (2 or 4 digits) appears once, indicating either YY or YYYY
   4. The string is not followed by \[word character or '/'\]

### Verification: 
   - Convert the Date column to 'datetime' data type. Strings successfully converted are valid dates.
   2. Check if there is at most one match for each entry.
   3. Record the number of dates filled. 


In [3]:
# Check if each entry only has one date
# match not followed by or preceded by'/'
# new_df = df['Text'].str.extractall(r'((?<![\w\/])(?:\d{1,2}\/){1,2}(?:\d{4}|\d{2})(?![\w\/]))') 
new_df = df['Text'].str.extractall(r'((?<![\w\/])(?:\d{1,2}\/){1,2}(?:\d{4}|\d{2})(?![\w\/]))') 
new_df.reset_index(inplace=True)
new_df.info() # 231 matches for 230 instances
new_df[new_df['match'] == 1]
# Return '16/22', the second match of Instance 10, and it's not a valid date.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231 entries, 0 to 230
Data columns (total 3 columns):
level_0    231 non-null int64
match      231 non-null int64
0          231 non-null object
dtypes: int64(2), object(1)
memory usage: 5.5+ KB


,level_0,match,0
11,10,1,16/22


In [4]:
# Only extract the first match if any, and create a new column 'Date'.
selection_1 = df['Text'].str.contains(r'((?<![\w\/])(?:\d{1,2}\/){1,2}(?:\d{4}|\d{2})(?![\w\/]))').copy()
df.loc[selection_1,'Date'] = df.loc[selection_1,'Text'].str.extract(r'((?<![\w\/])(?:\d{1,2}\/){1,2}(?:\d{4}|\d{2})(?![\w\/]))')

# Add '19' before the two-digit year for xx/xx/xx
df.loc[selection_1,'Date'] = df.loc[selection_1,'Date'].str.replace(r'((?<![\w\/])(?:\d{1,2}\/){1,2}\d{2}(?![\w\/]))', lambda x: x.group(0)[:-2]+'19'+x.group(0)[-2:])

# Verify the number of dates added
df.info() # 230 dates added, 270 remaining
df.head(10)

# Alternative methods using map or apply
# len(df[(df['Date'].map(type) == pd._libs.tslib.Timestamp) == True])
# df.apply(lambda x: type(x['Date']) == pd._libs.tslib.Timestamp,axis=1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 2 columns):
Text    500 non-null object
Date    230 non-null object
dtypes: object(2)
memory usage: 7.9+ KB


/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  from ipykernel import kernelapp as app
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  app.launch_new_instance()


,Text,Date
0,03/25/93 Total time of visit (in minutes):\n,03/25/1993
1,6/18/85 Primary Care Doctor:\n,6/18/1985
2,sshe plans to move as of 7/8/71 In-Home Servic...,7/8/1971
3,7 on 9/27/75 Audit C Score Current:\n,9/27/1975
4,2/6/96 sleep studyPain Treatment Pain Level (N...,2/6/1996
5,.Per 7/06/79 Movement D/O note:\n,7/06/1979
6,"4, 5/18/78 Patient's thoughts about current su...",5/18/1978
7,10/24/89 CPT Code: 90801 - Psychiatric Diagnos...,10/24/1989
8,3/7/86 SOS-10 Total Score:\n,3/7/1986
9,(4/10/71)Score-1Audit C Score Current:\n,4/10/1971


## 2. Extract Dates Formatted Month(alphabetic) Day Year

### Example: 

* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009

### Pattern: 
   - (An alphabetic character) appears at least 3 times, indicating the month
   2. [non-word character] appears at least once
   3. The following appears 0-1 times
      - (1-2 digits) appears once, followed by (2 alphabetic character) 0-1 times, indicating the day
      - [non-word character] appears at least once
   5. (4 digits) appears once, indicating the year

### Verification: 
   - Convert the Date column to 'datetime' data type. Strings successfully converted are valid dates.
   2. Check if there is at most one match for each entry.
   3. Search if there is overlapping with the dates already filled above
   4. Record the number of dates filled. 



In [5]:
# Check if each entry only has one date
new_df = df['Text'].str.extractall(r'((?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|(?:Nov|Dec)(?:ember)?)\W*\d{1,2}(?:st|nd|rd|th)?\W+(?:\d{4}|\d{2})(?!\d))')
# new_df = df['Text'].str.extractall(r'(\b[a-zA-Z]{3,}\b(?:\d{1,2}(?:st|nd|rd|th)?)?\b\d{2,4})')
new_df.reset_index(inplace=True)
new_df.info() # 34 matches for 34 instances
new_df[new_df['match'] == 1] # Return empty, meaning only one such match for each instance if any

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 3 columns):
level_0    34 non-null int64
match      34 non-null int64
0          34 non-null object
dtypes: int64(2), object(1)
memory usage: 896.0+ bytes


,level_0,match,0


In [6]:
# Check if there's overlapping with the first format.
selection_2 = df['Text'].str.contains(r'((?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|(?:Nov|Dec)(?:ember)?)\W*\d{1,2}(?:st|nd|rd|th)?\W+(?:\d{4}|\d{2})(?!\d))')
flag = True
for i in df.loc[selection_1].index:
    if i in df.loc[selection_2].index: 
        flag = False
        print('It overlaps with the first format.')
        break
if flag: print('It does not overlap with the first format.')
print('-'*40)
# Add dates
df.loc[selection_2,'Date'] = df.loc[selection_2,'Text'].str.extract(r'((?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|(?:Nov|Dec)(?:ember)?)\W*\d{1,2}(?:st|nd|rd|th)?\W+(?:\d{4}|\d{2})(?!\d))')

# Verify the number of dates added
df.info() # 34 dates added, 264 overall, 236 remaining
df.sample(20)

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  from ipykernel import kernelapp as app
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:12: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)


It does not overlap with the first format.
----------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 2 columns):
Text    500 non-null object
Date    264 non-null object
dtypes: object(2)
memory usage: 7.9+ KB


,Text,Date
343,6/1998 Primary Care Doctor:\n,6/1998
73,4/12/74= 1Caffeine / Tobacco Use Caffeinated p...,4/12/1974
382,sOne prior voluntary hospitalization in 09/197...,09/1975
334,"sAppendectomy in July 1986, shortly after deli...",NaN
121,10/02/96 Age:\n,10/02/1996
283,AFeb 1977: Symmes Hospital\n,NaN
276,April 1986 Primary Care Doctor:\n,NaN
483,sTexas Medical Center; Oklahoma for 2 weeks; 1...,NaN
352,See initial PROMPTCARE evaluation from 3/1986 ...,3/1986
443,s30 days in GA in 9/2000. Hx of Outpatient Tre...,9/2000


## 3. Extract Dates Formatted of Day Month(alphabetic) Year

### Example: 

* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009


### Pattern: 
   - (1-2 digits) appears once, followed by (2 alphabetic character) 0-1 times, indicating the day
   2. [non-word character] appears at least once
   3 (An alphabetic character) appears at least 3 times, indicating the month
   4. [non-word character] appears at least once
   5. (4 digits) appears once, indicating the year

### Verification: 
   - Convert the Date column to 'datetime' data type. Strings successfully converted are valid dates.
   2. Check if there is at most one match for each entry.
   3. Search if there is overlapping with the dates already filled above
   4. Record the number of dates filled. 

In [7]:
# Check if each entry only has one date
new_df = df['Text'].str.extractall(r'((?<![\d])\d{1,2}(?:st|nd|rd|th)?\W*(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|(?:Nov|Dec)(?:ember)?)\W*\d{4}(?!\d))')
# new_df = df['Text'].str.extractall(r'(\b[a-zA-Z]{3,}\b(?:\d{1,2}(?:st|nd|rd|th)?)?\b\d{2,4})')
new_df.reset_index(inplace=True)
new_df.info() # 70 matches for 70 instances
new_df[new_df['match'] == 1] # Return empty, meaning only one such match for each instance if any

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 3 columns):
level_0    70 non-null int64
match      70 non-null int64
0          70 non-null object
dtypes: int64(2), object(1)
memory usage: 1.7+ KB


,level_0,match,0


In [8]:
# Check if there's overlapping with the first format.
selection_3 = df['Text'].str.contains(r'((?<![\d])\d{1,2}(?:st|nd|rd|th)?\W*(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|(?:Nov|Dec)(?:ember)?)\W*\d{4}(?!\d))')
flag = True
for i in df.loc[selection_3].index:
    if i in df.loc[selection_1].index:
        flag = False
        print('It overlaps with the first format.')
        print(df.loc[i])
    if i in df.loc[selection_2].index: 
        flag = False
        print('It overlaps with the second format.')
        print(df.loc[i])
if flag: print('It does not overlap with the previous formats.')
print('-'*40)
# Add dates
df.loc[selection_3,'Date'] = df.loc[selection_3,'Text'].str.extract(r'((?<![\d])\d{1,2}(?:st|nd|rd|th)?\W*(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|(?:Nov|Dec)(?:ember)?)\W*\d{4}(?!\d))')

# Verify the number of dates added
df.info() # 70 dates added, 334 overall, 166 remaining
df.sample(20)

It does not overlap with the previous formats.
----------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 2 columns):
Text    500 non-null object
Date    334 non-null object
dtypes: object(2)
memory usage: 7.9+ KB


/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  from ipykernel import kernelapp as app
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:16: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)


,Text,Date
101,10/05/97 CPT Code: 90791: No medical services\n,10/05/1997
481,y1974 (all)\n,NaN
313,nLoss of father to cardiac event in Decemeber ...,NaN
442,9/1992 Primary Care Doctor:\n,9/1992
478,eHistory of small right parietal subgaleal hem...,NaN
156,26 May 1974 Primary Care Doctor:\n,26 May 1974
137,7intake for follow up treatment at Anson Gener...,10 Feb 1983
249,"0h/o 3 detoxes (2 for EtOH, 1 opioids). Alask...",NaN
248,";Trazodone 50-100 mg QHS, had also been on thi...",NaN
159,.The patient brought himself to APS on 26 May ...,26 May 2001


## 4. Extract Dates Formatted of Month(alphabetic) Year

### Example: 

* Feb 2009; Sep 2009; Oct 2010

Find these dates among the rest of the instances that haven't had any date value yet.

### Pattern: 
   - (An alphabetic character) appears at least 3 times, indicating the month
   4. [non-word character] appears at least once
   5. (4 digits) appears once, indicating the year


In [9]:
# Check if each entry only has one date
new_df = df.loc[df['Date'].isnull(),'Text'].str.extractall(r'(\b(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|(?:Nov|Dec)(?:ember)?)\W*\d{4}(?!\d))')
new_df.reset_index(inplace=True)
new_df.info() # 105 matches for 105 instances
new_df[new_df['match'] == 1] # Return empty, meaning only one such match for each instance if any

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 3 columns):
level_0    105 non-null int64
match      105 non-null int64
0          105 non-null object
dtypes: int64(2), object(1)
memory usage: 2.5+ KB


,level_0,match,0


In [10]:
# Add dates
df.loc[df['Date'].isnull(),'Date'] = df.loc[df['Date'].isnull(),'Text'].str.extract(r'(\b(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|(?:Nov|Dec)(?:ember)?)\W*\d{4}(?!\d))')
# Verify the number of dates added
df.info() # 105 dates added, 439 overall, 61 remaining
df.sample(20)

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  from ipykernel import kernelapp as app


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 2 columns):
Text    500 non-null object
Date    439 non-null object
dtypes: object(2)
memory usage: 7.9+ KB


,Text,Date
374,weight reduction surgery- bariatic sleeve 11/2...,11/2000
426,Retired in 11/1984 after teaching for 30 years...,11/1984
90,10/13/95 CPT Code: 90791: No medical services\n,10/13/1995
157,short term partial hospital program at Pine Re...,10 Feb 1990
427,6e. monitor pt's depressive experience in cont...,5/2016
286,father died suddenly in January 2013 Meaningfu...,January 2013
204,"Mar. 10, 1976 CPT Code: 90791: No medical serv...","Mar. 10, 1976"
437,bObesity : 11/1986 MCC LAPAROSCOPIC PARTIAL VE...,11/1986
137,7intake for follow up treatment at Anson Gener...,10 Feb 1983
274,"7HH, April 1985 Hx of Outpatient Treatment:\n",April 1985


## 5. Extract Dates in Format of Year

### Example: 

* 2009; 2010

Find these dates among the rest of the instances that haven't had any date value yet.

### Pattern: 

   - (4 digits) appears once, indicating the year
   - No digits precede or follow the sub-string

In [11]:
# Check if each entry only has one date
new_df = df.loc[df['Date'].isnull(),'Text'].str.extractall(r'((?<!\d)\d{4}(?!\d))')
new_df.reset_index(inplace=True)
new_df.info() # 56 matches for 56 instances
new_df[new_df['match'] == 1] # Return empty, meaning only one such match for each instance if any

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 3 columns):
level_0    56 non-null int64
match      56 non-null int64
0          56 non-null object
dtypes: int64(2), object(1)
memory usage: 1.4+ KB


,level_0,match,0


In [12]:
# Add dates
df.loc[df['Date'].isnull(),'Date'] = df.loc[df['Date'].isnull(),'Text'].str.extract(r'((?<!\d)\d{4}(?!\d))')
# Verify the number of dates added
df.info() # 56 dates added, 495 overall, 5 remaining
df.sample(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 2 columns):
Text    500 non-null object
Date    495 non-null object
dtypes: object(2)
memory usage: 7.9+ KB


/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  from ipykernel import kernelapp as app


,Text,Date
393,4/2012 open heart surgery\n,4/2012
357,% 10/1993 Echo: LVEF 60%\n,10/1993
237,"safter evicted in February 1976, hospitalized ...",February 1976
67,7/6/91 SOS-10 Total Score:\n,7/6/1991
98,5/13/72 Other Adult Mental Health Outcomes Sca...,5/13/1972
97,3/31/91 Communication with referring physician...,3/31/1991
233,"Dr. Gloria English, who conducted an initial c...","July, 1990"
195,"MRI May 30, 2001 empty sella but no problems w...","May 30, 2001"
10,"(5/11/85) Crt-1.96, BUN-26; AST/ALT-16/22; WBC...",5/11/1985
213,"July 11, 1997 CPT Code: 90792: With medical se...","July 11, 1997"


## 6. Deal with the Rest

There are still 5 instances left. 

In [13]:
df.loc[df['Date'].isnull()] # [25,39,40,71,99]

,Text,Date
25,4-13-82 Other Child Mental Health Outcomes Sca...,NaN
39,1-14-81 Communication with referring physician...,NaN
40,7-29-75 CPT Code: 90801 - Psychiatric Diagnosi...,NaN
71,4-13-89 Communication with referring physician...,NaN
99,011/14/83 Audit C Score Current:\n,NaN


In [14]:
df.loc[[25,39,40,71],'Date'] = df.loc[[25,39,40,71],'Text'].str[:5] + '19' + df.loc[[25,39,40,71],'Text'].str[5:7]
df.loc[99,'Date'] = df.loc[99,'Text'][1:7] + '19' + df.loc[99,'Text'][7:9]
df.info()
df.loc[[25,39,40,71,99]]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 2 columns):
Text    500 non-null object
Date    500 non-null object
dtypes: object(2)
memory usage: 7.9+ KB


,Text,Date
25,4-13-82 Other Child Mental Health Outcomes Sca...,4-13-1982
39,1-14-81 Communication with referring physician...,1-14-1981
40,7-29-75 CPT Code: 90801 - Psychiatric Diagnosi...,7-29-1975
71,4-13-89 Communication with referring physician...,4-13-1989
99,011/14/83 Audit C Score Current:\n,11/14/1983


In [15]:
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

In [16]:
df[df['Date'].isnull()]

,Text,Date
321,"2June, 1999 Audit C Score Current:\n",NaT


In [17]:
df.loc[321,'Date'] = pd.to_datetime('2 June, 1999')
df.loc[321,'Date']

Timestamp('1999-06-02 00:00:00')

In [18]:
df.sort_values(by='Date')

,Text,Date
9,(4/10/71)Score-1Audit C Score Current:\n,1971-04-10
84,5/18/71 Total time of visit (in minutes):\n,1971-05-18
2,sshe plans to move as of 7/8/71 In-Home Servic...,1971-07-08
53,7/11/71 SOS-10 Total Score:\n,1971-07-11
28,9/12/71 [report_end]\n,1971-09-12
474,sPatient reported losing three friends that pa...,1972-01-01
153,13 Jan 1972 Primary Care Doctor:\n,1972-01-13
13,1/26/72 Communication with referring physician...,1972-01-26
129,06 May 1972 SOS-10 Total Score:\n,1972-05-06
98,5/13/72 Other Adult Mental Health Outcomes Sca...,1972-05-13


## Put Everything Together

In [19]:
def date_sorter():
    
    doc = []
    with open('dates.txt') as file:
        for line in file:
            doc.append(line)
    df = pd.Series(doc)
    df = df.to_frame().rename(columns={0:'Text'})
    
    selection_1 = df['Text'].str.contains(r'((?<![\w\/])(?:\d{1,2}\/){1,2}(?:\d{4}|\d{2})(?![\w\/]))')
    df.loc[selection_1,'Date'] = df.loc[selection_1,'Text'].str.extract(r'((?<![\w\/])(?:\d{1,2}\/){1,2}(?:\d{4}|\d{2})(?![\w\/]))')
    # Add '19' before the two-digit year for xx/xx/xx
    df.loc[selection_1,'Date'] = df.loc[selection_1,'Date'].str.replace(r'((?<![\w\/])(?:\d{1,2}\/){1,2}\d{2}(?![\w\/]))', lambda x: x.group(0)[:-2]+'19'+x.group(0)[-2:])
    
    selection_2 = df['Text'].str.contains(r'((?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|(?:Nov|Dec)(?:ember)?)\W*\d{1,2}(?:st|nd|rd|th)?\W+(?:\d{4}|\d{2})(?!\d))')
    df.loc[selection_2,'Date'] = df.loc[selection_2,'Text'].str.extract(r'((?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|(?:Nov|Dec)(?:ember)?)\W*\d{1,2}(?:st|nd|rd|th)?\W+(?:\d{4}|\d{2})(?!\d))')

    selection_3 = df['Text'].str.contains(r'((?<![\d])\d{1,2}(?:st|nd|rd|th)?\W*(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|(?:Nov|Dec)(?:ember)?)\W*\d{4}(?!\d))')
    df.loc[selection_3,'Date'] = df.loc[selection_3,'Text'].str.extract(r'((?<![\d])\d{1,2}(?:st|nd|rd|th)?\W*(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|(?:Nov|Dec)(?:ember)?)\W*\d{4}(?!\d))')
    
    df.loc[df['Date'].isnull(),'Date'] = df.loc[df['Date'].isnull(),'Text'].str.extract(r'(\b(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|(?:Nov|Dec)(?:ember)?)\W*\d{4}(?!\d))')
    
    df.loc[df['Date'].isnull(),'Date'] = df.loc[df['Date'].isnull(),'Text'].str.extract(r'((?<!\d)\d{4}(?!\d))')
    
    df.loc[[25,39,40,71],'Date'] = df.loc[[25,39,40,71],'Text'].str[:5] + '19' + df.loc[[25,39,40,71],'Text'].str[5:7]
    df.loc[99,'Date'] = df.loc[99,'Text'][1:7] + '19' + df.loc[99,'Text'][7:9]
    
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    df.loc[321,'Date'] = pd.to_datetime('2 June, 1999')
    
    ans = pd.Series(df.sort_values(by='Date').index)
    return ans

In [20]:
date_sorter()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:10: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:11: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:15: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:16: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:18: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
/opt/conda/lib/python3.6/site-packag

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     405
20     323
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    286
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    439
486    255
487    401
488    475
489    235
490    257
491    152
492    464
493    253
494    427
495    231
496    141
497    186
498    161
499    413
Length: 500, dtype: int64